In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sklearn
import seaborn as sns
import re
import sklearn
import seaborn as sns
import re
from collections import Counter
%matplotlib inline

In [2]:
# Import the Data Set
data = pd.read_csv('https://github.com/Thinkful-Ed/data-201-resources/raw/master/hotel-reviews.csv')

In [4]:
# Perform some basic cleaning and character removal

# Make everything lower case
data['reviews.text'] = data['reviews.text'].str.lower()

# Remove non-text characters
data['reviews.text'] = data['reviews.text'].str.replace(r'\.|\!|\?|\'|,|-|\(|\)', "",)

# Fill in black reviews with '' rather than Null (which would give us errors)
data['reviews.text'] = data['reviews.text'].fillna('')

'''
Basic rules to natural language processing. we are looking to find words with the strongest meanings
'''
# print data.dtypes
# print data['address'].value_counts()
    

In [5]:
'''
you lose meaning by vectorizing. it can be a weakness. What about relative frequency? or context?
number of times out of total number of words that the word appears- rel. freq.

use rel. freq. because it normalizes the data

to deal with anomalies or outliers, make sure you understand the distribution of your data. you make 
the decision of whether or not it is worth keeping outliers.

to maintain context, consider using n-grams or using each feature as a set of words tather than 
individual words. https://www.wikiwand.com/en/N-gram

'''

# Import and initiate a vectorizer
from sklearn.feature_extraction.text import CountVectorizer
# The max features is how many words we want to allow us to create columns for
vectorizer = CountVectorizer(max_features=5000)
#so do we only get 5000 words? or what does 5000
#only count up to 5000 words. limit num of columns created
#coding the words into 1's and 0's 
#one word could be (00010)
#another could be (100000)

In [6]:
# Vectorize our reviews to transform sentences into columns
X = vectorizer.fit_transform(data['reviews.text'])

# And then put all of that in a table
bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [7]:
# Rename some columns for clarity
data.rename(columns={'address': 'hotel_address', 'city': 'hotel_city',
                     'country':'hotel_country', 'name':'hotel_name'},
            inplace=True)

# Join our bag of words back to our initial hotel data
full_df = data.join(bag_of_words)

In [8]:
# X is our words
X = bag_of_words

# Y is our hotel name (the outcome we care about)
Y_hotel = data['hotel_name']

In [9]:
# Import a random forest model
'''
you can use Nbayes, logistic regression, or any other word to vector type models
model --> word2vec (tensorflow)/RNN (gives position/spacing of words)
text --> gensim
'''

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

# Fit that random forest model to our data
rfc.fit(X,Y_hotel)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [10]:
# Write your own dream vacation review here...
test_review = ['''
    I loved the beach and the sunshine and the clean and modern room.
    ''']

In [11]:
# Convert your test review into a vector
X_test = vectorizer.transform(test_review).toarray()

In [12]:
# Match your review
prediction = rfc.predict(X_test)[0]

In [14]:
# Return the essential information about your match
data[data['hotel_name'] == prediction][['hotel_name', 'hotel_address', 
                                        'hotel_city', 'hotel_country']].head(1)

,hotel_name,hotel_address,hotel_city,hotel_country
33706,Casa Del Mar Inn Bandb,18 Bath St,Santa Barbara,US
